In [1]:
import os
from glob import glob
import yaml

from jinja2 import Template

import nbformat as nbf
import papermill as pm

In [2]:
from papermill.engines import NBClientEngine

class CustomEngine(NBClientEngine):
    @classmethod
    def execute_managed_notebook(cls, nb_man, kernel_name, **kwargs):
        jinja_data = {} if "jinja_data" not in kwargs else kwargs["jinja_data"]

        # call the papermill execution engine:
        super().execute_managed_notebook(nb_man, kernel_name, **kwargs)

        for cell in nb_man.nb.cells:
            if cell.cell_type == "markdown":
                cell["source"] = Template(cell["source"]).render(**jinja_data)


# what's the right way to register an engine?
pm.engines.papermill_engines._engines["md_jinja"] = CustomEngine
pm.engines.papermill_engines._engines

{None: papermill.engines.NBClientEngine,
 'nbclient': papermill.engines.NBClientEngine,
 'md_jinja': __main__.CustomEngine}

In [3]:
control = yaml.safe_load(
    """
output_dir: ./output
kernel_name: metabolic
notebooks:
    hello-world:
      parameter_groups:
        a=10_b=1:
          a: 10
          b: 1
    notebook2:
      kernel_name: sno
      parameter_groups:
        temperature:
          variable_id: TEMP
        salinity:
          variable_id: SALT
        oxygen:
          variable_id: O2
    """)
control

{'output_dir': './output',
 'kernel_name': 'metabolic',
 'notebooks': {'hello-world': {'parameter_groups': {'a=10_b=1': {'a': 10,
     'b': 1}}},
  'notebook2': {'kernel_name': 'sno',
   'parameter_groups': {'temperature': {'variable_id': 'TEMP'},
    'salinity': {'variable_id': 'SALT'},
    'oxygen': {'variable_id': 'O2'}}}}}

In [4]:
output_dir = "." if 'output_dir' not in control else control["output_dir"]
os.makedirs(output_dir, exist_ok=True)

if 'kernel_name' in control:
    for d in control['notebooks'].values():
        if 'kernel_name' not in d:
            d['kernel_name'] = control['kernel_name']

In [5]:
files = glob(f"{output_dir}/*.ipynb")
for f in files:
    os.remove(f)

In [6]:

for nb, info in control['notebooks'].items():
    for key, parms in info['parameter_groups'].items():
        
        input_path = f'{nb}.ipynb'        
        output_path = f'{output_dir}/{nb}-{key}.ipynb'
        
        o = pm.execute_notebook(
            input_path=input_path,
            output_path=output_path,
            kernel_name=info['kernel_name'],
            parameters=parms,
            engine_name='md_jinja',
            jinja_data=parms,
        )

Executing:   0%|          | 0/8 [00:00<?, ?cell/s]

Executing:   0%|          | 0/4 [00:00<?, ?cell/s]

Executing:   0%|          | 0/4 [00:00<?, ?cell/s]

Executing:   0%|          | 0/4 [00:00<?, ?cell/s]